In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler
import warnings

c:\Users\Kyuho\anaconda3\envs\helix\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
SEED = 0
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)

TRAIN_RATIO = 0.8
BATCH_SIZE = 512
LEARNING_RATE = 0.0001
# LEARNING_RATE = 0.001
ENCODER_EPOCH = 100
DECODER_EPOCH = 0
REPORT_PATH = 'experiments.txt'

LAYER_ARR = [200, 400, 800, 1600, 1600, 1600, 800, 400, 400, 400, 200, 100]
KLD_WEIGHT = 1e-6

In [3]:
# mixture
# DATASET_PATH = 'non_gaussian_tracks\\tracks_1M_gaussian_mixturev2.txt'
# VAL_DATASET_PATH = 'non_gaussian_tracks\\tracks_100k_gaussian_mixturev2.txt'
# ENCODER_RESULTS_PATH = 'variance_predict\\mixture\\results3\\'
# DECODER_RESULTS_PATH = 'variance_predict\\mixture\\results3\\'

DATASET_PATH = 'C:\\Users\\Kyuho\\code\\UCI\\helix\\tracks_100k_updated_test.txt'
ENCODER_RESULTS_PATH = 'variance_predict\\gaussian\\vae_results1\\'

# DATASET_PATH = 'tracks_1m_updated_asymmetric_higher.txt'
# VAL_DATASET_PATH = 'tracks_100k_updated_asymmetric_higher.txt'
# ENCODER_RESULTS_PATH = 'separated\\asymmetric\\results1\\'
# DECODER_RESULTS_PATH = 'separated\\asymmetric\\results1\\'

MODEL_PATH = 'C:\\Users\\Kyuho\\code\\UCI\\helix\\variance_predict\\gaussian\\vae_results1\\encoder_epoch_85.pth'

In [4]:
class Dataset(Dataset):
    def __init__(self, path, transform=None):
        with open(path, 'r') as file:
            content = file.read()
            data_points = content.split('EOT')

            data_points = [dp.strip() for dp in data_points if dp.strip()]
            data_points = [dp.split('\n') for dp in data_points]
            data_points = [[[float(cell) for cell in row.split(', ')] for row in dp] for dp in data_points]
            self.original_targets = np.array([dp[0] for dp in data_points])
            input_points = [dp[1:] for dp in data_points]
            self.scaler = MinMaxScaler()
            self.original_targets_rescaled = self.scaler.fit_transform(self.original_targets)
            self.original_targets_tensor = torch.tensor(self.original_targets)
            inputs = []
            for input in input_points:
                combined = []
                for coordinate in input:
                    combined += coordinate
                inputs.append(combined)
            self.inputs = torch.tensor(np.array(inputs))
            # self.inputs = torch.tensor(np.array(input_points))

    def __len__(self):
        return len(self.original_targets_rescaled)

    def __getitem__(self, idx):
        target = self.original_targets_tensor[idx]
        input = self.inputs[idx]
        return input, target

In [5]:
def find_phi_inverse(target_radius_squared, d0, phi0, pt, dz, tanl, eps=1e-6):
    alpha = 1 / 2  # 1/cB
    q = 1
    kappa = q / pt
    rho = alpha / kappa

    calculated_term_x = d0 * torch.cos(phi0) + rho * torch.cos(phi0) 
    calculated_term_y = d0 * torch.sin(phi0) + rho * torch.sin(phi0)

    hypotenuse = torch.sqrt(calculated_term_x ** 2 + calculated_term_y ** 2)

    arccos_input = (calculated_term_x ** 2 + calculated_term_y ** 2 + rho ** 2 - target_radius_squared) / (2 * rho * hypotenuse)
    clamped = torch.clamp(arccos_input, min=-1 + eps, max=1-eps) # to avoid nan
    arccos_term = torch.acos(clamped)
    phi = arccos_term   

    return phi % (2 * torch.pi) # wrap angle

def compute_3d_track(phi, d0, phi0, pt, dz, tanl):
    alpha = 1 / 2  # 1/cB
    q = 1
    kappa = q / pt
    rho = alpha / kappa
    x = d0 * torch.cos(phi0) + rho * (torch.cos(phi0) - torch.cos(phi0 + phi))
    y = d0 * torch.sin(phi0) + rho * (torch.sin(phi0) - torch.sin(phi0 + phi))
    z = dz - rho * tanl * phi
    return x, y, z
    
# Generate noiseless hit points in 3D using optimized phi values
def generate_noiseless_hits(params, min_radius=1.0, max_radius=10.0, num_layers=10):

    d0, phi0, pt, dz, tanl = params[:, 0], params[:, 1], params[:, 2], params[:, 3], params[:, 4]
    xs, ys, zs = [], [], []
    target_radii = [torch.tensor(r, requires_grad=True) for r in torch.linspace(min_radius, max_radius, num_layers)]
    #for target_radius in torch.linspace(min_radius, max_radius, num_layers, device=device):
    for target_radius in target_radii:
        # Optimize phi for the current radius
        phi = find_phi_inverse(target_radius**2, d0, phi0, pt, dz, tanl)
        x, y, z = compute_3d_track(phi, d0, phi0, pt, dz, tanl)
        xs.append(x)
        ys.append(y)
        zs.append(z)

    # Stack results into a tensor
    hit_points = torch.stack((torch.stack(xs, dim=1), torch.stack(ys, dim=1), torch.stack(zs, dim=1)), dim=2)

    #return hit_points, total_distance / num_layers, torch.stack(optimized_phi_values)
    return hit_points

In [6]:
class GenerateNoiselessHitsLayer(nn.Module):
    def __init__(self):
        super(GenerateNoiselessHitsLayer, self).__init__()

    def forward(self, z):
        return generate_noiseless_hits(z)

class VAE_Encoder(nn.Module):
    def __init__(self, input_size=30, hidden_sizes=LAYER_ARR, latent_size=5):
        super().__init__()
        layer_sizes = [input_size] + hidden_sizes
        self.hidden_layers = nn.ModuleList([nn.Linear(layer_sizes[i], layer_sizes[i + 1]) for i in range(len(layer_sizes) - 1)])
        self.fc_mu = nn.Linear(hidden_sizes[-1], latent_size)
        self.fc_logvar = nn.Linear(hidden_sizes[-1], latent_size)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):

        # print("x input was ", x)
        for layer in self.hidden_layers:
            x = torch.relu(layer(x))
        mu, logvar = self.fc_mu(x), self.fc_logvar(x)
        # print("mu before was ", mu[0])
        # print("logvar before was ", logvar[0])
        z = self.reparameterize(mu, logvar)
        # print("z after reparameterize ", z[0])
        return z, mu, logvar


class VAE_EncoderWithHits(nn.Module):
    def __init__(self, input_size=30, hidden_sizes=LAYER_ARR, latent_size=5):
        super().__init__()
        self.encoder = VAE_Encoder(input_size, hidden_sizes, latent_size)
        self.noiseless_hits_layer = GenerateNoiselessHitsLayer()

    def forward(self, x):
        z, mu, logvar = self.encoder(x)   
        logvar = torch.clamp(logvar, min=-10, max=10)
        # print("z was ", z[0])
        reconstructed_hits = self.noiseless_hits_layer(z)
        # print("reconstructed_hits was ", reconstructed_hits[0])
        return reconstructed_hits, mu, logvar

In [7]:
def compute_differences_batch(set1, set2):
    c2 = torch.sum((set1 - set2) ** 2, dim=(1, 2))
    return torch.mean(c2)

def compute_differences_no_mean(set1, set2):
    return torch.sum((set1 - set2) ** 2, dim=(1, 2))

In [8]:
# load test data
# load model
# run through the test data
# compute 3d for both original parameter and the predicted parameter
# get the distance
def run_encoder(encoder, encoder_optimizer, dataloader, device, data_size, prev_encoder_path =''):
    print(f"Loading model from {prev_encoder_path}")
    checkpoint = torch.load(prev_encoder_path)
    encoder.load_state_dict(checkpoint['model_state_dict'])
    encoder_optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    distances = torch.zeros(data_size)
    current_size = 0
    for points, params in dataloader:
        points = points.float().to(device)
        params = params.float().to(device)
        reconstructed_points, mean, logvar = encoder(points)
        input_points = points.view(points.shape[0], 10, 3)
        distance = compute_differences_no_mean(reconstructed_points, input_points)
        distances[current_size:current_size + distance.shape[0]] = distance
        current_size += distance.shape[0]
        print(current_size)

    return distances



In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
dataset = Dataset(DATASET_PATH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
encoder = VAE_EncoderWithHits()
encoder = encoder.to(device)
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr = LEARNING_RATE)
encoder_scheduler = None

all_distances = run_encoder(encoder, encoder_optimizer, dataloader, device, len(dataset), MODEL_PATH)

Using device: cuda
Loading model from C:\Users\Kyuho\code\UCI\helix\variance_predict\gaussian\vae_results1\encoder_epoch_85.pth


C:\Users\Kyuho\AppData\Local\Temp\ipykernel_22304\543877620.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(prev_encoder_path)
C:\Users\Kyuho\App

512
1024
1536
2048
2560
3072
3584
4096
4608
5120
5632
6144
6656
7168
7680
8192
8704
9216
9728
10240
10752
11264
11776
12288
12800
13312
13824
14336
14848
15360
15872
16384
16896
17408
17920
18432
18944
19456
19968
20480
20992
21504
22016
22528
23040
23552
24064
24576
25088
25600
26112
26624
27136
27648
28160
28672
29184
29696
30208
30720
31232
31744
32256
32768
33280
33792
34304
34816
35328
35840
36352
36864
37376
37888
38400
38912
39424
39936
40448
40960
41472
41984
42496
43008
43520
44032
44544
45056
45568
46080
46592
47104
47616
48128
48640
49152
49664
50176
50688
51200
51712
52224
52736
53248
53760
54272
54784
55296
55808
56320
56832
57344
57856
58368
58880
59392
59904
60416
60928
61440
61952
62464
62976
63488
64000
64512
65024
65536
66048
66560
67072
67584
68096
68608
69120
69632
70144
70656
71168
71680
72192
72704
73216
73728
74240
74752
75264
75776
76288
76800
77312
77824
78336
78848
79360
79872
80384
80896
81408
81920
82432
82944
83456
83968
84480
84992
85504
86016
86528
87040


In [10]:
torch.mean(all_distances)

tensor(2.3419, grad_fn=<MeanBackward0>)

In [11]:
torch.max(all_distances)

tensor(1393.9694, grad_fn=<MaxBackward1>)

In [12]:
torch.min(all_distances)

tensor(0.0913, grad_fn=<MinBackward1>)

In [13]:
all_distances_list = all_distances.tolist()
sorted_all_distances_list = sorted(all_distances_list)

In [14]:
sorted_all_distances_list[-7]

31.22881507873535

In [15]:
import matplotlib.pyplot as plt

In [16]:
# plt.hist(all_distances_list, bins=10, edgecolor='black')
# plt.title('Data Distribution')
# plt.xlabel('Value')
# plt.ylabel('Frequency')
# plt.show()

In [20]:
less_1 = 0
less_4 = 0
less_10 = 0
less_100 = 0
remaining = 0

for distance in all_distances_list:
    if distance < 1:
        less_1 += 1
    elif distance < 4:
        less_4 += 1
    elif distance < 10:
        less_10 += 1
    elif distance < 100:
        less_100 += 1
    else:
        remaining += 1
    

In [21]:
print(less_1, less_4, less_10, less_100, remaining)

36951 46728 14224 2095 2
